In [7]:
# This script is to generate a list of all the targets observed with
# their frame number and dither positions (based on YOFFSETS)
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
import sys
import glob
import pandas as pd


''' NIRES header keywords of interest
Dither pattern name: DPATNAME
Dither pattern step size in X: DPATSTPX
Dither pattern step size in Y: DPATSTPY
Current frame dither position X offset: XOFFSET
Current frame dither position Y offset: YOFFSET
Pixel scale: PSCALE
ITIME: Integration time
DATAFILE: image saved data file name
OBJECT: object name
TARGNAME: target name
OBSTYPE: observation type
'''


date ='20181126'
# import all the science and telluric frames from one observation
frame_list = glob.glob('NIRES/' + date + '/' + 'Science&Telluric/s*.fits')

all_headers = []
for frame in range(len(frame_list)):
    hdul = fits.open(frame_list[frame])
    all_headers.append(hdul[0].header)
    hdul.close()

dither_pattern = []
framenum = []
targname = []
for i in range(len(all_headers)):
    if 'J' in all_headers[i]['TARGNAME']:
        framenum.append(all_headers[i]['FRAMENUM'])
        targname.append(all_headers[i]['TARGNAME'])
        if all_headers[i]['YOFFSET'] == -5:
            dither_pattern.append('A')
        elif all_headers[i]['YOFFSET'] == 0:
            dither_pattern.append('B')
        elif all_headers[i]['YOFFSET'] == 5:
            dither_pattern.append('C')
            
    elif 'H' in all_headers[i]['TARGNAME']:
        framenum.append(all_headers[i]['FRAMENUM'])
        targname.append(all_headers[i]['TARGNAME'])
        dither_pattern.append(all_headers[i]['YOFFSET'])
data = {'FRAMENUM':framenum,'TARGNAME':targname,'DITHER':dither_pattern}

df = pd.DataFrame(data)
df = df.sort_values(by='FRAMENUM')
df.to_csv(f'NIRES_PypeIt/{date}_reduced/{date}_ditherpattern.csv',index = False)